In [ ]:
import math
import requests
import numpy as np
import pandas as pd
import altair as alt
import xml.etree.ElementTree as ET

In [ ]:
alt.data_transformers.enable('json')

In [ ]:
url = 'https://www.ecb.europa.eu/stats/eurofxref/eurofxref-hist.xml'

In [ ]:
response = requests.get(url)

In [ ]:
tree = ET.fromstring(response.content)

In [ ]:
rates = []
for date in list(tree[2]):
    for rate in list(date):
        rates.append({'date': date.attrib['time'], 'currency': rate.attrib['currency'], 'rate': rate.attrib['rate']})

In [ ]:
df = pd.DataFrame(rates)
df = df.astype(dtype={'date': 'datetime64', 'currency': 'object', 'rate': 'float64'})
df = df.sort_values(['date', 'currency'], ascending=[True, True]).reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
data = df[df['currency'].isin(['SGD', 'CNY'])]

_domain_y = (math.floor(data['rate'].min()), math.ceil(data['rate'].max()))

alt.Chart(data).mark_line().encode(
    alt.X('date:T'),
    alt.Y('rate:Q', scale=alt.Scale(domain=_domain_y)),
    alt.Color('currency')
).properties(
    width=720,
    height=400
).configure_axis(
    grid=False
).configure_view(
    strokeOpacity=0
).interactive()